## Nightingale's coxcombs reimagined

Florence Nightingale was not only the founder of modern nursing, but also a pioneering statistican who used infographic visualizations commonly referred to as "coxcombs" to make a case for improving the sanitation conditions of hospitals. A Nightingale coxcomb is composed of 12 equal-degree slices of a disc, one corresponding to each month. Each slice shows the proportions deaths due to different causes. Each colour of wedge within a slice represents a different cause of death. In Nightingale's original diagram below, the blue wedge's areas represents deaths from preventable or zymotic diseases. The area of the red wedge represents deaths from wounds and injuries, and the area of the black wedge represents deaths from all other causes.


![title](coxcombs.jpg)

The intention of Nightingale's coxcombs diagram was to communicate **the rate of soldiers dying from preventable zymotic diseases** by juxtaposing the proportions of these deaths against those due to other causes of war. The following dashboard redesign of Nightingale's coxcombs preserves this intention.

----

Firstly, using the **annual mortality rate** measurement proposed by Nightingale, we will examine **the difference between annual mortality rates by month, based on cause of mortality**. This measure is represented by the following formula, where *D* represents the number of mortalities that month due to the cause examined, *S* is the average size of army that month, and the 12,000 numerator represents rate per thousand, across 12 months: 

$$\frac{1200D}{S} = \text{Annual Mortality Rate (AMR)}$$

One challenge of the original coxcombs graphic is comparing proportions across time. The circular diagram is meant to communicate the cycle of the year and makes seasonal comparison easy, however, because only 12 months are fitted at a time on the disc, this makes comparisons across years, or even across non-consecutive months, a challenge. 

----

My dashboard redesign aims to: 
1. facilitate easier comparison across different months, years 
2. to illustrate direct comparisons between aggregate Annual Mortality Rates, including and excluding those due to zymotic disease.

In [45]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [49]:
#reading in both short and long versions of data
df = pd.read_csv('Nightingale.csv')
df_long = pd.read_csv('Nightingalelong.csv')

In [53]:
#create bar graph animation showing annual rates of mortality, by cause, by month
arm_over_time = px.bar(df_long, x="Cause of Death", y="Annual rate of mortality",color="Cause of Death",
  animation_frame="Month", range_y=[0,1100], title="Annual Rates of Mortality by Cause", 
  labels={"Annual rate of mortality":"Annual rate of mortality (based on monthly rate)","Cause of Death":"Cause of Mortality"})
arm_over_time.show()

By following the changing rates across months, one can see that the annual rates of mortality associated with preventable zymotic diseases is consistently higher than those associated with other causes of death which are non-preventable. Notably, starting in April 1855 after additional sanitation measures are introduced, the annual rate of mortality by zymotic diseases decreases.
In order to again emphasize the proportion of preventable, zymotic disease-associated deaths, as done in the month-by-month time lapse annual rate of mortality measures above, it would also be of use to see these patterns over the timespan of data collection. In order to compare the deaths that are preventable against those that are not, two new aggregate measures are introduced: the monthly annual rate of mortality from all causes of death, and the monthly annual rate of mortality from all non-preventable causes of death.

In [50]:
#aggregates annual mortality rates due to zymotic disease, wounds & injuries, & other causes into one value for each month
def aggregate_deaths (row):
    death_total = row["Zymotic diseases AMR"] + row["Wounds & injuries AMR"] + row["All other causes AMR"]
    return death_total
#aggregates annual mortality rates due to wounds & injuries & other causes, into one value for each month
def aggregate_deaths_non_preventable (row):
    death_total = row["Wounds & injuries AMR"] + row["All other causes AMR"]
    return death_total


In [51]:
df["All Deaths"] = df.apply(lambda row: aggregate_deaths(row), axis=1)
df["All Deaths without Preventable"] = df.apply(lambda row: aggregate_deaths_non_preventable(row), axis=1)

In [52]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df["Month"],y=df["All Deaths"], name="AMR")
)
fig.add_trace(
    go.Scatter(x=df["Month"],y=df["All Deaths without Preventable"], name="AMR, excluding zymotic", line=dict(dash="dash"))
)
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="category"
    ),
    #sets default of hover to list all variables
    hovermode='x',
    title="Annual Mortality Rates by Month",
    yaxis_title="Annual Mortality Rate"
)
fig.add_vline(x="April 1855",line_width=1, line_dash="dash",)

These diagrams allow for focused comparisons to be made between the estimated annual rates of mortality based on monthly death toll proportions, with the impacts of zymotic diseases and without.
The dashed line for April 1855 represents the beginning of improvements to hospital conditions, and the ensuing annual mortality rate falling. 
This emphasizes the impacts that sanitary conditions had on enabling a much lower AMR that more closely matches that annual mortality rate if zymotic disease-related deaths are at 0. 
Using the range selector the viewer may adjust the period of time to more closely examine the drastic difference between the real and hypothetical AMRs, for example in January of 1855, at the peak of zymotic-disease associated deaths. Showing that these lines later on almost converge, illustrates that gaps in care were manufacturing a much more severe death toll. With basic improvements, the AMR due to zymotic disease is visibly reduced to the goal of 0 deaths due to zymotic disease.
In creating this juxtaposition, it is my aim to emphasize Nightingale's persuasive argument for basic improvements to sanitary conditions.

## References

http://understandinguncertainty.org/node/213

http://understandinguncertainty.org/node/214

https://timelyportfolio.github.io/rCharts_micropolar/nightingale/index.html

http://www.florence-nightingale-avenging-angel.co.uk/?page_id=2382